#### Librerias necesarias

In [13]:
import os 
import tensorflow as tf 
from tensorflow.keras import regularizers
from tensorflow.keras import layers 
from tensorflow.keras import Model 
from tensorflow import keras
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import matplotlib.image as mpimg
from glob import glob 
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.preprocessing.image import ImageDataGenerator


#### Directorio de las imagenes:

In [10]:
dir = r'C:\Users\Admin\Desktop\Proyecto\TB_Chest_Radiography_Database'
train_dir = os.path.join(dir, 'train')
test_dir = os.path.join(dir, 'test')
train_normal_dir = os.path.join(train_dir, 'Normal')
train_tb_dir = os.path.join(train_dir, 'Tuberculosis')
test_normal_dir = os.path.join(test_dir, 'Normal')
test_tb_dir = os.path.join(test_dir, 'Tuberculosis')

#train_n_fnames = os.listdir(train_normal_dir)
#test_tb_fnames = os.listdir(test_tb_dir)


In [ ]:
nrows = 4
ncols = 4
pic_index = 0
fig = plt.gcf()
fig.set_size_inches(ncols*4, nrows*4)
pic_index += 8
next_n_pix = [os.path.join(train_normal_dir, fname)
            for fname in train_normal_dir[pic_index-8:pic_index] 
            ]

next_tb_pix = [os.path.join(train_tb_dir, fname)
            for fname in train_tb_dir[pic_index-8:pic_index] 
            ]            

for i, img_path in enumerate(next_n_pix+next_tb_pix):
    sp = plt.subplot(nrows, ncols, i+1)
    sp.axis('off')
    img = mpimg.imread(img_path)
    plt.imshow()

## Creacion de la red neuronal:
#### se usa la funcion de perdida SparseCategoricalCrossentropy, a normal = 0 y tb = 1

In [ ]:
class convu(tf.keras.Model):
    def __init__(self, filters, kernel_size):
        super(Convu, self).__init__()

        self.conv = tf.keras.layers.Conv2D(filters=filters, kernel_size=kernel_size)
        self.bn = tf.keras.layers.BatchNormalization()
        self.relu = tf.keras.layers.ReLU()
        self.pool = tf.keras.layers.MaxPool2D(pool_size=(2, 2))

    def call(self, inputs, training=True):
        x = self.conv(inputs)
        x = self.bn(x, training=training)
        x = self.relu(x)
        x = self.pool(x)
        return(x)


model = tf.keras.Sequential(name='rnc_rayos_x')
model.add(convu(filters=32, kernel_size=(3, 3)))
model.add(convu(filters=64, kernel_size=(3, 3)))
model.add(convu(filters=128, kernel_size=(3, 3)))
model.add(convu(filters=128, kernel_size=(3, 3)))
model.add(tf.keras.layers.Flatten())
model.add(tf.keras.layers.Dense(units=512, activation=tf.keras.activations.relu))
model.add(tf.keras.layers.Dense(units=2, activation=tf.keras.activations.softmax))
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-4), loss=tf.keras.losses.SparseCategoricalCrossentropy(), metrics=['accuracy'])

dir = r'C:\Users\Admin\Desktop\Proyecto\TB_Chest_Radiography_Database'
train_len = len(glob(os.path.join(dir, 'train', 'Normal', '*.png')))*2
test_len = len(glob(os.path.join(dir, 'test', 'Normal', '*.png')))*2

In [ ]:
train_datagen = ImageDataGenerator(rescale = 1.0/255, rotation_range=40, width_shift_range=0.2, height_shift_range=0.2, shear_range=0.2, horizontal_flip=True, fill_mode='nearest')
test_datagen = ImageDataGenerator(rescale = 1.0/255, rotation_range=40, width_shift_range=0.2, height_shift_range=0.2, shear_range=0.2, horizontal_flip=True, fill_mode='nearest')
train_generador = train_datagen.flow_from_directory(train_dir, batch_size=20, class_mode='binary', target_size=(150, 150))
test_generador = test_datagen.flow_from_directory(test_dir, batch_size=20, class_mode='binary', target_size=(150, 150))
checkpoint_path = "training_1/cp.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)

callback = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_path, save_best_only=True, save_weights_only=False, verbose=1)

### Se crean funciones para la manipulación de imágenes:

In [ ]:
def load(a, label):
    # cargar el archivo en un tensor:
    image = tf.io.read_file(a)
    # decodificando en formato JPEG
    image = tf.image.decode_jpeg(image)
    # convertir a tf.float32 
    image = tf.cast(image, tf.float32)
    return(image, label)


def resize(input_image, size):
    return(tf.image.random_crop(input_image, size))

